In [1]:
# -*- coding: utf-8 -*-
"""
모든 CSV에서 '거래일자' <= 2015-10-19 행 드랍하고 새 폴더에 저장
- 인코딩: utf-8-sig -> cp949 -> utf-8 순서로 시도
- 날짜 파싱: 포맷 섞여 있어도 to_datetime(errors='coerce')로 강건 처리
- '거래일자' 공백/변형 명도 감지 (예: ' 거래일자 ', '거래 일자')
- NaT(파싱실패)는 보존, 파싱된 값 중 <= 2015-10-19 만 제거
"""

from pathlib import Path
import pandas as pd
import re

# ----------------- 경로 설정 (여기만 본인 환경에 맞게 수정) -----------------
INPUT_DIR = Path("/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/전처리 최종")
# -----------------------------------------------------------------------------

CUTOFF = pd.Timestamp("2015-10-19")
OUTPUT_DIR = INPUT_DIR.with_name(INPUT_DIR.name + "_after_2015-10-19")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def read_csv_any(p: Path) -> pd.DataFrame:
    errors = []
    for enc in ("utf-8-sig", "cp949", "utf-8"):
        try:
            return pd.read_csv(p, encoding=enc)
        except Exception as e:
            errors.append(f"{enc}: {e}")
    raise RuntimeError(f"[읽기 실패] {p.name}\n" + "\n".join(errors))

def find_date_col(df: pd.DataFrame) -> str:
    # 공백 제거 후 비교
    canon = {re.sub(r"\s+", "", str(c)): c for c in df.columns}
    for key in ("거래일자", "일자", "날짜"):
        if key in canon:
            return canon[key]
    # 한국어 컬럼명이 미묘하게 다른 경우(공백/탭 등)도 한 번 더 탐색
    for c in df.columns:
        if re.sub(r"\s+", "", str(c)) in {"거래일자"}:
            return c
    raise KeyError("이 파일에 '거래일자' 컬럼이 없습니다.")

def process_file(csv_path: Path) -> dict:
    df = read_csv_any(csv_path)
    date_col = find_date_col(df)

    # 날짜 파싱 (실패는 NaT)
    dt = pd.to_datetime(df[date_col], errors="coerce")

    # 드랍 마스크: 파싱 성공 & dt <= CUTOFF
    drop_mask = dt.notna() & (dt <= CUTOFF)
    kept_mask = ~drop_mask  # NaT 포함 보존

    before_n = len(df)
    after_n = int(kept_mask.sum())
    removed_n = int(drop_mask.sum())

    out_df = df.loc[kept_mask].copy()

    # 출력 파일 경로
    out_path = OUTPUT_DIR / csv_path.name
    out_df.to_csv(out_path, index=False, encoding="utf-8-sig")

    # 요약 정보
    min_after = pd.to_datetime(out_df[date_col], errors="coerce").min()
    return {
        "file": csv_path.name,
        "before": before_n,
        "removed": removed_n,
        "after": after_n,
        "min_date_after": None if pd.isna(min_after) else str(min_after.date()),
        "output": str(out_path),
    }

def main():
    csv_files = sorted([p for p in INPUT_DIR.glob("*.csv") if p.is_file()])
    if not csv_files:
        raise SystemExit(f"[에러] CSV를 찾지 못했습니다: {INPUT_DIR}")

    summary = []
    print(f"[작업 시작] 입력 폴더: {INPUT_DIR}")
    print(f"[저장 폴더] {OUTPUT_DIR}\n")

    for p in csv_files:
        try:
            info = process_file(p)
            summary.append(info)
            print(f" - {p.name}: {info['removed']}행 제거 → {info['after']}행 저장 | "
                  f"최소 날짜(보존): {info['min_date_after']}")
        except Exception as e:
            print(f"[스킵/에러] {p.name}: {e}")

    # 요약 표
    if summary:
        print("\n[요약]")
        cols = ["file", "before", "removed", "after", "min_date_after", "output"]
        df_sum = pd.DataFrame(summary, columns=cols)
        # 보기 좋게 정렬
        df_sum = df_sum.sort_values("file").reset_index(drop=True)
        # 콘솔 출력
        with pd.option_context('display.max_colwidth', 120):
            print(df_sum.to_string(index=False))

if __name__ == "__main__":
    main()


[작업 시작] 입력 폴더: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/전처리 최종
[저장 폴더] /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/전처리 최종_after_2015-10-19

 - 감자_cleaned.csv: 112행 제거 → 3542행 저장 | 최소 날짜(보존): 2015-10-20
 - 고구마_cleaned.csv: 224행 제거 → 7084행 저장 | 최소 날짜(보존): 2015-10-20
 - 깐마늘_통합_최종_데이터.csv: 94행 제거 → 2957행 저장 | 최소 날짜(보존): 2015-10-20
 - 깻잎_cleaned.csv: 112행 제거 → 3542행 저장 | 최소 날짜(보존): 2015-10-20
 - 느타리버섯_cleaned.csv: 336행 제거 → 10626행 저장 | 최소 날짜(보존): 2015-10-20
 - 단호박_cleaned.csv: 112행 제거 → 3542행 저장 | 최소 날짜(보존): 2015-10-20
 - 당근_cleaned.csv: 112행 제거 → 3542행 저장 | 최소 날짜(보존): 2015-10-20
 - 미나리_cleaned.csv: 224행 제거 → 7084행 저장 | 최소 날짜(보존): 2015-10-20
 - 부추_cleaned.csv: 224행 제거 → 7084행 저장 | 최소 날짜(보존): 2015-10-20
 - 브로콜리_cleaned.csv: 336행 제거 → 10626행 저장 | 최소 날짜(보존): 2015-10-20
 - 새송이버섯_cleaned.csv: 112행 제거 → 3542행 저장 | 최소 날짜(보존): 2015-10-20
 - 시금치_cleaned.csv: 112행 제거 → 3542행 저장 | 최소 날짜(보존): 2015-10-20
 - 쑥갓_cleaned.csv: 0행 제거 → 0행 저장 | 최소 날짜(보존): None
 - 애호박_cleaned.csv: 224행 제거 

In [4]:
# -*- coding: utf-8 -*-
"""
'원본 전처리 최종' 폴더 안 CSV 파일들의 행(row), 열(column) 개수 확인
"""

from pathlib import Path
import pandas as pd

# ----------------- 경로 설정 -----------------
FOLDER = Path("/Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종")
# ---------------------------------------------

def get_shape_info(folder: Path):
    files = sorted([p for p in folder.glob("*.csv") if p.is_file()])
    if not files:
        raise SystemExit(f"[에러] CSV 파일이 없습니다: {folder}")

    summary = []
    for f in files:
        try:
            df = pd.read_csv(f, encoding="utf-8-sig")
        except UnicodeDecodeError:
            df = pd.read_csv(f, encoding="cp949")
        rows, cols = df.shape
        summary.append({
            "file": f.name,
            "rows": rows,
            "cols": cols,
        })

    return pd.DataFrame(summary)

def main():
    print(f"[대상 폴더] {FOLDER}\n")
    df_info = get_shape_info(FOLDER)
    print(df_info.to_string(index=False))

if __name__ == "__main__":
    main()


[대상 폴더] /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/원본 전처리 최종

                  file  rows  cols
        감자_cleaned.csv  3542    12
       고구마_cleaned.csv  7084    12
     깐마늘_통합_최종_데이터.csv  2957    12
        깻잎_cleaned.csv  3542    12
     느타리버섯_cleaned.csv 10626    12
       단호박_cleaned.csv  3542    12
        당근_cleaned.csv  3542    12
       미나리_cleaned.csv  7084    12
        부추_cleaned.csv  7084    12
      브로콜리_cleaned.csv 10626    12
     새송이버섯_cleaned.csv  3542    12
       시금치_cleaned.csv  3542    12
       애호박_cleaned.csv  7084    12
       양배추_cleaned.csv  7084    12
       양상추_cleaned.csv  7084    12
     양송이버섯_cleaned.csv  3542    12
        양파_cleaned.csv 10626    12
        오이_cleaned.csv 10626    12
     적양배추_filtered.csv  3542    12
        쪽파_cleaned.csv 10626    12
청고추(풋고추)_통합_최종_데이터.csv  2957    12
          청고추_머지제외.csv 14168    12
       치커리_cleaned.csv 10626    12
     토마토전체_cleaned.csv 14168    12
         파프리카_머지제외.csv  7084    12
  파프리카녹색_통합_최종_데이터